In [1]:
import tensorflow as tf 
import keras 
import numpy as np 
import pandas as pd

In [2]:
min_time_steps = 4
max_time_steps = 9
Input_Dimension = 10
train_size = 4000
test_size = 400

In [3]:
def make_random () :
    return [np.random.randint(1,9) for i in range(np.random.randint(min_time_steps,max_time_steps))]

In [85]:
X = [make_random() for i in range(6000)]
y =[make_random()for i in range(6000)]

In [5]:
max_input_seq = max(len(seq) for seq in X)
max_output_seq = max(len(seq) for seq in y)

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [86]:
x_train = pad_sequences(X,padding='post',maxlen=max_input_seq,truncating='pre')
y_train = pad_sequences(y,padding='post',maxlen=max_input_seq,truncating='pre')

In [8]:
from keras.layers import Masking 

In [9]:
masking = Masking(mask_value=[1,0,0,0,0,0,0,0,0,0])

In [87]:
encoder_input = x_train.copy()
decoder_output = y_train.copy()
decoder_input = decoder_output.copy()

In [11]:
decoder_input[0] 

array([4, 2, 7, 7, 6, 7, 0, 0])

In [16]:
y = pad_sequences(y,padding='post',truncating='pre',maxlen=max_output_seq)

In [88]:
for i in range(len(decoder_input)) :
    decoder_input[i] = np.roll(decoder_input[i],shift=1)
    decoder_input[i][0] = 0

In [89]:
encoder_input = keras.utils.to_categorical(encoder_input,num_classes=Input_Dimension)
decoder_output = keras.utils.to_categorical(decoder_output,num_classes=Input_Dimension)
decoder_input = keras.utils.to_categorical(decoder_input,num_classes=Input_Dimension)

In [90]:
max_input_seq

8

In [91]:
encoder_Input = keras.Input(shape=(max_input_seq,Input_Dimension))
encoder_masked = Masking(mask_value=[1,0,0,0,0,0,0,0,0,0])(encoder_Input)
encoder_Lstm = keras.layers.LSTM(128,return_state=True)
output_encoder,state_h,state_c = encoder_Lstm(encoder_masked)
encoder_states = [state_h,state_c]
decoder_Input = keras.Input(shape=(None,Input_Dimension))
decoder_LStm = keras.layers.LSTM(128,return_state=True,return_sequences=True)
output_decoder,_,_= decoder_LStm(decoder_Input,initial_state=encoder_states)
decoder_dense = keras.layers.Dense(10,activation='softmax')
output_decoder = decoder_dense(output_decoder)

In [92]:
encoder_decoder = keras.Model([encoder_Input,decoder_Input],output_decoder)

In [93]:
encoder_decoder.compile(optimizer=keras.optimizers.AdamW(learning_rate=0.01),loss='categorical_crossentropy',metrics=["accuracy"])

In [94]:
encoder_decoder.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 8, 10)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, 8, 10)     │          0 │ input_layer_14[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_8 (Masking) │ (None, 8, 10)     │          0 │ input_layer_14[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_3 (Any)         │ (None, 8)         │          0 │ not_equal_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_15      │ (None, None, 10)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_14 (LSTM)      │ [(None, 128),     │     71,168 │ masking_8[0][0],  │
│                     │ (None, 128),      │            │ any_3[0][0]       │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_15 (LSTM)      │ [(None, None,     │     71,168 │ input_layer_15[0… │
│                     │ 128), (None,      │            │ lstm_14[0][1],    │
│                     │ 128), (None,      │            │ lstm_14[0][2]     │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None, 10)  │      1,290 │ lstm_15[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 143,626 (561.04 KB)

 Trainable params: 143,626 (561.04 KB)

 Non-trainable params: 0 (0.00 B)

In [98]:
encoder_decoder.fit([encoder_input,decoder_input],decoder_output,verbose=2,epochs=20)

Epoch 1/20
188/188 - 3s - 18ms/step - accuracy: 0.8615 - loss: 0.4042
Epoch 2/20
188/188 - 3s - 17ms/step - accuracy: 0.8649 - loss: 0.3943
Epoch 3/20
188/188 - 3s - 17ms/step - accuracy: 0.8737 - loss: 0.3723
Epoch 4/20
188/188 - 3s - 18ms/step - accuracy: 0.8709 - loss: 0.3788
Epoch 5/20
188/188 - 3s - 17ms/step - accuracy: 0.8686 - loss: 0.3845
Epoch 6/20
188/188 - 3s - 18ms/step - accuracy: 0.8633 - loss: 0.3982
Epoch 7/20
188/188 - 3s - 18ms/step - accuracy: 0.8657 - loss: 0.3905
Epoch 8/20
188/188 - 3s - 17ms/step - accuracy: 0.8730 - loss: 0.3696
Epoch 9/20
188/188 - 4s - 19ms/step - accuracy: 0.8752 - loss: 0.3657
Epoch 10/20
188/188 - 3s - 17ms/step - accuracy: 0.8790 - loss: 0.3537
Epoch 11/20
188/188 - 3s - 18ms/step - accuracy: 0.8762 - loss: 0.3654
Epoch 12/20
188/188 - 3s - 18ms/step - accuracy: 0.8774 - loss: 0.3573
Epoch 13/20
188/188 - 3s - 17ms/step - accuracy: 0.8769 - loss: 0.3600
Epoch 14/20
188/188 - 5s - 28ms/step - accuracy: 0.8796 - loss: 0.3543
Epoch 15/20
188

In [99]:
encoder_decoder.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 8, 10)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, 8, 10)     │          0 │ input_layer_14[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_8 (Masking) │ (None, 8, 10)     │          0 │ input_layer_14[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_3 (Any)         │ (None, 8)         │          0 │ not_equal_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_15      │ (None, None, 10)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_14 (LSTM)      │ [(None, 128),     │     71,168 │ masking_8[0][0],  │
│                     │ (None, 128),      │            │ any_3[0][0]       │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_15 (LSTM)      │ [(None, None,     │     71,168 │ input_layer_15[0… │
│                     │ 128), (None,      │            │ lstm_14[0][1],    │
│                     │ 128), (None,      │            │ lstm_14[0][2]     │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None, 10)  │      1,290 │ lstm_15[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 430,880 (1.64 MB)

 Trainable params: 143,626 (561.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 287,254 (1.10 MB)

In [100]:
encoder_models = keras.Model(encoder_Input,encoder_states)

In [102]:
encoder_models.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 8, 10)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, 8, 10)     │          0 │ input_layer_14[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_8 (Masking) │ (None, 8, 10)     │          0 │ input_layer_14[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_3 (Any)         │ (None, 8)         │          0 │ not_equal_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_14 (LSTM)      │ [(None, 128),     │     71,168 │ masking_8[0][0],  │
│                     │ (None, 128),      │            │ any_3[0][0]       │
│                     │ (None, 128)]      │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 71,168 (278.00 KB)

 Trainable params: 71,168 (278.00 KB)

 Non-trainable params: 0 (0.00 B)

In [103]:
decoder_input_state_h = keras.Input(shape=(128,))
decoder_input_state_c = keras.Input(shape=(128,))
decoder_input_all_states = [decoder_input_state_h,decoder_input_state_c]
decoder_outputs,state_h,state_c = decoder_LStm(decoder_Input,initial_state=decoder_input_all_states)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)

In [104]:
decoder_models = keras.Model(inputs=[decoder_Input] + decoder_input_all_states,
                             outputs=[decoder_outputs] + decoder_states)

In [105]:
decoder_models.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15      │ (None, None, 10)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_16      │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_17      │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_15 (LSTM)      │ [(None, None,     │     71,168 │ input_layer_15[0… │
│                     │ 128), (None,      │            │ input_layer_16[0… │
│                     │ 128), (None,      │            │ input_layer_17[0… │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None, 10)  │      1,290 │ lstm_15[1][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 72,458 (283.04 KB)

 Trainable params: 72,458 (283.04 KB)

 Non-trainable params: 0 (0.00 B)

In [114]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_models.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, Input_Dimension))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1 
    # START (0 zero) in one-hot-encoding --> [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_seq = list()
    while not stop_condition:

        # in a loop
        # decode the input to a token/output prediction + required states for context vector
        output_tokens, h, c = decoder_models.predict(
            [target_seq] + states_value)

        # convert the token/output prediction to a token/output
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = sampled_token_index
        # add the predicted token/output to output sequence
        decoded_seq.append(sampled_char)
        

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 0 or
           len(decoded_seq) == max_output_seq):
            stop_condition = True

        # Update the input target sequence (of length 1) 
        # with the predicted token/output 
        target_seq = np.zeros((1, 1, Input_Dimension))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update input states (context vector) 
        # with the ouputed states
        states_value = [h, c]

        # loop back.....
        
    # when loop exists return the output sequence
    return decoded_seq

In [145]:
target_testing = list()
for i in range(100) :
    target_testing.append(encoder_input[np.random.randint(0,6000)])

In [154]:
target_testing[0]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [159]:
y_pred = list()
for i in range(100) :
    target_testing = np.array([encoder_input[np.random.randint(0,6000)]])
    y_pred.append(target_testing)

In [160]:
y_pred

[array([[[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]),
 array([[[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]),
 array([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1.,

In [157]:
target_testing

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]])

In [ ]:
y_predict = list() 
for i in range(len(y_pred)) :
    predicted = decode_sequence(input_seq=y_pred[i])
    y_predict.append(predicted)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━